# Using My Blog to Rewrite Itself

Here I use my blog made of Jupyter notebooks to rewrite itself.

In [100]:
#| default_exp main

In [101]:
#| export
from datetime import datetime
from execnb.nbio import read_nb
from nb2fasthtml.core import render_code_output
from fasthtml.common import *
from fasthtml.jupyter import *
from IPython.display import display, HTML
from monsterui import franken
from monsterui.all import Theme
import mistletoe
import pygments
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

In [102]:
tags = ["Python", "Jupyter"]

## Background

Before this notebook, the code for my blog was a main.py that I maintained by editing it in VS Code. The index page and notebooks were rendered without MonsterUI. I wanted to keep it minimal because sometimes I do crazy CSS experiments.

Yesterday in [Customizing FastHTML Headers From Notebook Contents](https://audrey.feldroy.com/nbs/2025-01-22-Customizing-FastHTML-Headers-From-Notebook-Contents) I discovered that different routes can have different HTML headers thanks to `_xt_cts`. I then modified my notebook detail view to include MonsterUI headers only when I tag the notebook with "MonsterUI".

Today I was inspired by [Solve It With Code](https://solveit.fast.ai/) Lesson 11 to try rewriting more of my blog in MonsterUI. 

## Setup

In [103]:
#| export
app,rt = fast_app(pico=False)

In [104]:
server = JupyUvi(app)

## Notebook Index: Cards

In [105]:
def NBCard(t,d):
    return franken.Card(d, header=franken.H3(t))

In [106]:
NBCard("First Post", "This is a description of what my post is about")

```html
<div class="uk-card ">
  <div class="uk-card-header ">
    <h3 class="uk-h3 ">First Post</h3>
  </div>
  <div class="uk-card-body space-y-6">This is a description of what my post is about</div>
</div>

```

In [107]:
show(NBCard("First Post", "This is a description of what my post is about"))

In [153]:
#| export
def get_nb_paths(): return L(sorted(Path().glob("nbs/*.ipynb"), reverse=True))

Note: change to `glob("*.ipynb")` to run this from the notebook

In [154]:
nb_paths = get_nb_paths()
nb_paths

(#0) []

In [110]:
read_nb(nb_paths[0]).cells[0].source.lstrip("# ")

'Using My Blog to Rewrite Itself'

This is the title of the first notebook.

In [111]:
#| export
def get_title_and_desc(fpath):
    nb = read_nb(fpath)
    title = nb.cells[0].source.lstrip("# ")
    desc = nb.cells[1].source
    return title,desc

In [112]:
get_title_and_desc(nb_paths[0])

('Using My Blog to Rewrite Itself',
 'Here I use my blog made of Jupyter notebooks to rewrite itself.')

In [113]:
show(NBCard(*get_title_and_desc(nb_paths[0])))

In [114]:
def mk_nbcard_from_nb_path(nb_path):
    return NBCard(*get_title_and_desc(nb_path))

In [115]:
mk_nbcard_from_nb_path(nb_paths[0])

```html
<div class="uk-card ">
  <div class="uk-card-header ">
    <h3 class="uk-h3 ">Using My Blog to Rewrite Itself</h3>
  </div>
  <div class="uk-card-body space-y-6">Here I use my blog made of Jupyter notebooks to rewrite itself.</div>
</div>

```

In [116]:
nb_paths.map(mk_nbcard_from_nb_path)[:2]

(#2) [div((div((h3(('Using My Blog to Rewrite Itself',),{'class': 'uk-h3 '}),),{'class': 'uk-card-header '}), div(('Here I use my blog made of Jupyter notebooks to rewrite itself.',),{'class': 'uk-card-body space-y-6'})),{'class': 'uk-card '}),div((div((h3(('Troubleshooting MonsterUI on This Site',),{'class': 'uk-h3 '}),),{'class': 'uk-card-header '}), div(("My first MonsterUI notebook isn't rendering correctly. Here I debug it.",),{'class': 'uk-card-body space-y-6'})),{'class': 'uk-card '})]

In [117]:
show(*nb_paths.map(mk_nbcard_from_nb_path)[:2])

## Notebook Index: Add Dates to Cards

In [155]:
#| export
def get_date_from_iso8601_prefix(fname):
    "Gets date from first 10 chars YYYY-MM-DD of `fname`, where `fname` is like `2025-01-12-Get-Date-From-This.whatever"
    try:
        return datetime.fromisoformat(str(fname)[4:14])
    except ValueError: return None

In [119]:
nb_paths[0]

Path('2025-01-23-Using-My-Blog-to-Rewrite-Itself.ipynb')

In [120]:
date = get_date_from_iso8601_prefix(nb_paths[0])

In [121]:
Div(f"{date:%a, %b %-d, %Y}", style="font-size: 0.875rem;color:#666;")

```html
<div style="font-size: 0.875rem;color:#666;">Thu, Jan 23, 2025</div>

```

In [122]:
franken.PSmall(f"{date:%a, %b %-d, %Y}", cls="uk-text-muted")

```html
<p class="uk-text-small uk-text-muted">Thu, Jan 23, 2025</p>

```

In [123]:
def NBCard(t,d):
    return franken.Card(
        franken.CardTitle(franken.H3(t)), 
        franken.PSmall(f"{date:%a, %b %-d, %Y}", cls="uk-text-muted"),
        franken.P(d),
        body_cls='space-y-2'
    )

In [124]:
c = mk_nbcard_from_nb_path(nb_paths[0])
c

```html
<div class="uk-card ">
  <div class="uk-card-body space-y-2">
    <div class="uk-card-title ">
      <h3 class="uk-h3 ">Using My Blog to Rewrite Itself</h3>
    </div>
    <p class="uk-text-small uk-text-muted">Thu, Jan 23, 2025</p>
    <p>Here I use my blog made of Jupyter notebooks to rewrite itself.</p>
  </div>
</div>

```

In [125]:
show(c)

## Notebook Index: Linkify Cards

In [126]:
#| export
def NBCard(title,desc,href,date):
    return A(
        franken.Card(
        franken.CardTitle(franken.H3(title)), 
        franken.PSmall(f"{date:%a, %b %-d, %Y}", cls="uk-text-muted"),
        franken.P(desc),
        body_cls='space-y-2'
    ), href=href)

In [127]:
#| export
def mk_nbcard_from_nb_path(nb_path):
    date = get_date_from_iso8601_prefix(nb_path)
    return NBCard(*get_title_and_desc(nb_path), href=f'/nbs/{nb_path.name[:-6]}', date=date)

In [128]:
c = mk_nbcard_from_nb_path(nb_paths[1])
c

```html
<a href="/nbs/2025-01-23-Troubleshooting-MonsterUI-on-This-Site">  <div class="uk-card ">
    <div class="uk-card-body space-y-2">
      <div class="uk-card-title ">
        <h3 class="uk-h3 ">Troubleshooting MonsterUI on This Site</h3>
      </div>
      <p class="uk-text-small uk-text-muted">Thu, Jan 23, 2025</p>
      <p>My first MonsterUI notebook isn&#x27;t rendering correctly. Here I debug it.</p>
    </div>
  </div>
</a>
```

In [129]:
show(c)

## Notebook Index Page

In [130]:
#| export
@rt
def index():
    nb_paths = get_nb_paths()
    return (
        Theme.blue.headers(),
        franken.Container(
            Div(
                franken.H1('audrey.feldroy.com'), franken.PParagraph("The experimental Jupyter notebooks of Audrey M. Roy Greenfeld. This website and all its notebooks are open-source at ", franken.A("github.com/audreyfeldroy/arg-blog-fasthtml", href="https://github.com/audreyfeldroy/arg-blog-fasthtml"), cls="mb-6"),
            ),
            franken.Grid(*nb_paths.map(mk_nbcard_from_nb_path), cols_sm=1, cols_md=1, cols_lg=2, cols_xl=3)
        )
    )

## Notebook Detail: Frontmatter

In [131]:
t,d = get_title_and_desc(nb_paths[1])
t,d

('Troubleshooting MonsterUI on This Site',
 "My first MonsterUI notebook isn't rendering correctly. Here I debug it.")

In [132]:
mistletoe.markdown(d)

"<p>My first MonsterUI notebook isn't rendering correctly. Here I debug it.</p>\n"

## Notebook Detail: Non-Frontmatter

In [133]:
nb_paths[1]

Path('2025-01-23-Troubleshooting-MonsterUI-on-This-Site.ipynb')

### Use StyledCode (Pygments) for Code Cells

In [134]:
nb = read_nb(nb_paths[1])

In [135]:
nb.cells[5]

```json
{ 'cell_type': 'code',
  'execution_count': 3,
  'id': 'cb344dfb',
  'idx_': 5,
  'metadata': {},
  'outputs': [ { 'data': { 'text/markdown': [ '```html\n',
                                              '<h1 class="uk-h1 ">Hi</h1>\n',
                                              '\n',
                                              '```'],
                           'text/plain': ["h1(('Hi',),{'class': 'uk-h1 '})"]},
                 'execution_count': 3,
                 'metadata': {},
                 'output_type': 'execute_result'}],
  'source': 'H1("Hi")'}
```

In [136]:
#| export
def StyledCode(c, style='monokai'):
    fm = HtmlFormatter(style=style, cssclass=style)
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f".{style}")
    return Div(Style(sd), NotStr(h), id=style)

In [137]:
HTML(to_xml(StyledCode(nb.cells[2].source)))

In [138]:
HTML(to_xml(StyledCode(nb.cells[5].source)))

### Use Mistletoe for Markdown Cells

In [139]:
nb.cells[4]

```json
{ 'cell_type': 'markdown',
  'id': 'b91fdff1',
  'idx_': 4,
  'metadata': {},
  'source': '[MonsterUI Buttons and '
            'Links](https://audrey.feldroy.com/nbs/2025-01-22-MonsterUI-Buttons-and-Links) '
            'is the notebook with the problem.\n'
            '\n'
            'Let\'s start with a subset of the problem: the H1 text "MonsterUI '
            'Buttons and Links" is showing up as normal text.'}
```

In [140]:
#| export
def StyledMd(m):
    return Safe(mistletoe.markdown(m))

In [141]:
HTML(StyledMd(nb.cells[4].source))

### Use Mistletoe for Output Data of Code Cells

In [142]:
HTML(to_xml(StyledMd(nb.cells[5].outputs[0].data["text/markdown"])))

In [143]:
#| export
def StyledCell(c):
    if c.cell_type == "markdown": return StyledMd(c.source)
    if c.cell_type == "code": 
        if not c.outputs: return StyledCode(c.source)
        return StyledCode(c.source), render_code_output(c)

In [144]:
L(nb.cells).map(StyledCell)

(#7) ['<h1>Troubleshooting MonsterUI on This Site</h1>\n',"<p>My first MonsterUI notebook isn't rendering correctly. Here I debug it.</p>\n",div((style(('pre { line-height: 125%; }\ntd.linenos .normal { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }\nspan.linenos { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }\ntd.linenos .special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }\nspan.linenos.special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }\n.monokai .hll { background-color: #49483e }\n.monokai { background: #272822; color: #F8F8F2 }\n.monokai .c { color: #959077 } /* Comment */\n.monokai .err { color: #ED007E; background-color: #1E0010 } /* Error */\n.monokai .esc { color: #F8F8F2 } /* Escape */\n.monokai .g { color: #F8F8F2 } /* Generic */\n.monokai .k { color: #66D9EF } /* Keyword */\n.monokai .l { color: #AE81FF } /* Li

In [145]:
show(*L(nb.cells).map(StyledCell))

## Notebook Detail Page

In [147]:
#| export
@rt("/nbs/{name}")
def notebook(name:str):
    fpath = Path(f'nbs/{name}.ipynb')
    nb = read_nb(fpath)
    title = nb.cells[0].source.lstrip("# ")
    desc = nb.cells[1].source
    if "MonsterUI" in title:
        return (
            Theme.slate.headers(),
            franken.Container(
                franken.H1(title), # Title
                franken.P(desc), # Desc
                *L(nb.cells[2:]).map(StyledCell),
                cls="space-y-5"
            )
    )
    return (
        Style(':root {font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", sans-serif; color-scheme: light dark;} body {background-color: light-dark(#ffffff, #1a1a1a); color: light-dark(#1a1a1a, #ffffff);} p {line-height: 1.5;}'),
        Div(
            H1(title), # Title
            P(desc), # Desc
            *L(nb.cells[2:]).map(StyledCell),
            cls="space-y-5"
        )
    )

http://localhost:8000/nbs/2025-01-23-Troubleshooting-MonsterUI-on-This-Site

In [182]:
name = "2025-01-23-Troubleshooting-MonsterUI-on-This-Site"
fpath = Path(f'{name}.ipynb')
fpath.absolute()

Path('/Users/arg/fun/arg-blog-fasthtml/nbs/2025-01-23-Troubleshooting-MonsterUI-on-This-Site.ipynb')

## Troubleshooting the Troubleshooting MonsterUI Notebook

In [300]:
def is_code_cell(c): return c.cell_type == "code"
cells = L(read_nb(nb_paths[1]).cells).filter(is_code_cell)
cells

(#3) [{'cell_type': 'code', 'execution_count': 2, 'id': '8bff65e3', 'metadata': {}, 'outputs': [], 'source': 'from fasthtml.common import *\nfrom monsterui.all import *', 'idx_': 2},{'cell_type': 'code', 'execution_count': 3, 'id': 'cb344dfb', 'metadata': {}, 'outputs': [{'data': {'text/markdown': ['```html\n', '<h1 class="uk-h1 ">Hi</h1>\n', '\n', '```'], 'text/plain': ["h1(('Hi',),{'class': 'uk-h1 '})"]}, 'execution_count': 3, 'metadata': {}, 'output_type': 'execute_result'}], 'source': 'H1("Hi")', 'idx_': 5},{'cell_type': 'code', 'execution_count': 4, 'id': 'b86c1d3c', 'metadata': {}, 'outputs': [{'data': {'text/markdown': ['```html\n', '<a href="https://audrey.feldroy.com">My Blog</a>\n', '```'], 'text/plain': ["a(('My Blog',),{'href': 'https://audrey.feldroy.com'})"]}, 'execution_count': 4, 'metadata': {}, 'output_type': 'execute_result'}], 'source': 'A("My Blog", href="https://audrey.feldroy.com")', 'idx_': 6}]

In [301]:
cells[1]

```json
{ 'cell_type': 'code',
  'execution_count': 3,
  'id': 'cb344dfb',
  'idx_': 5,
  'metadata': {},
  'outputs': [ { 'data': { 'text/markdown': [ '```html\n',
                                              '<h1 class="uk-h1 ">Hi</h1>\n',
                                              '\n',
                                              '```'],
                           'text/plain': ["h1(('Hi',),{'class': 'uk-h1 '})"]},
                 'execution_count': 3,
                 'metadata': {},
                 'output_type': 'execute_result'}],
  'source': 'H1("Hi")'}
```

In [302]:
cells[1].outputs[0].data

```json
{ 'text/markdown': ['```html\n', '<h1 class="uk-h1 ">Hi</h1>\n', '\n', '```'],
  'text/plain': ["h1(('Hi',),{'class': 'uk-h1 '})"]}
```

## Troubleshooting the Cosine Similarity Notebook

In [296]:
def is_code_cell(c): return c.cell_type == "code"
cells = L(read_nb(nb_paths[9]).cells).filter(is_code_cell)
cells[0]

```json
{ 'cell_type': 'code',
  'execution_count': 7,
  'id': '06004824',
  'idx_': 6,
  'metadata': {},
  'outputs': [ { 'data': { 'text/latex': [ '$$\\text{cos}(\\theta) = '
                                           '\\frac{\\mathbf{A} \\cdot '
                                           '\\mathbf{B}}{\\|\\mathbf{A}\\| '
                                           '\\|\\mathbf{B}\\|}$$\n'],
                           'text/plain': [ '<IPython.core.display.Latex '
                                           'object>']},
                 'metadata': {},
                 'output_type': 'display_data'}],
  'source': '%%latex\n'
            '$$\\text{cos}(\\theta) = \\frac{\\mathbf{A} \\cdot '
            '\\mathbf{B}}{\\|\\mathbf{A}\\| \\|\\mathbf{B}\\|}$$'}
```

## Rendering Code Outputs With render_code_output

In [304]:
cells = L(read_nb(nb_paths[1]).cells).filter(is_code_cell)
cells[1]

```json
{ 'cell_type': 'code',
  'execution_count': 3,
  'id': 'cb344dfb',
  'idx_': 5,
  'metadata': {},
  'outputs': [ { 'data': { 'text/markdown': [ '```html\n',
                                              '<h1 class="uk-h1 ">Hi</h1>\n',
                                              '\n',
                                              '```'],
                           'text/plain': ["h1(('Hi',),{'class': 'uk-h1 '})"]},
                 'execution_count': 3,
                 'metadata': {},
                 'output_type': 'execute_result'}],
  'source': 'H1("Hi")'}
```

In [306]:
render_code_output(cells[1])

```html
<footer><pre><code class="language-html">&lt;h1 class="uk-h1 "&gt;Hi&lt;/h1&gt;

</code></pre>
</footer>

```

In [156]:
# show(*L(read_nb(nb_paths[4]).cells).map(StyledCell))

## Serve

In [ ]:
#| export
serve()

In [57]:
server.stop()

## Export

To export this notebook as arg-blog-fasthtml's main.py:

```bash
nb_export 2025-01-23-Using-My-Blog-to-Rewrite-Itself.ipynb
mv main.py ..
```